# Using LangChain with GPT4All and Custom Prompts

## References

1. [LangChain docs on using the GPT4All library](https://python.langchain.com/docs/integrations/llms/gpt4all)

## Code

### Import libraries

In [1]:
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

### Setup

In [2]:
local_path = r"C:\Users\deshi\Code\gpt4all-models\mistral-7b-instruct-v0.1.Q4_0.gguf"
# local_path = r"C:\Users\deshi\Code\gpt4all-models\orca-mini-3b-gguf2-q4_0.gguf"

In [11]:
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
llm = GPT4All(
    model=local_path, 
    callbacks=callbacks, 
    verbose=True,
    # max_tokens=1024,
    # n_predict=1024,
    # n_batch=4,
    # use_mlock=True,
    n_threads=8
)

### Standard template

In [ ]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
llmChain = LLMChain(prompt=prompt, llm=llm)

### Regular Inference

In [ ]:
question = """
Write a program to calculate the nth fibonacci number given a user input integer n. Also, print out all the n fibonacci numbers leading upto n in a list. Format the output in markdown format so it is easy to export to a markdown document.
"""

llmChain.run(question)

### Function Documentation Inference with Standard Template

In [ ]:
testFunction = """
n = int(input("Enter an integer: "))
result = 0
previous_two = 1
while result < n:
    next_two = previous_two + previous_two
    result += next_two
    previous_two = next_two
    print("The nth Fibonacci number is:", result)
"""

question = f"""
Here's my function in Python:\

{testFunction}\

Given the definition of a function in any programming language (particularly Python and C++), please generate it's stand-alone documentation. I want it complete with fields like function name, function arguments and return values as well as a detailed explanation of how the function logic works line-by-line. Make it concise and informative to put the documentation into a project documentation file.
"""

In [ ]:
llmChain.run(question)

### Custom Prompt Template

In [21]:
testFunction = """
def get_llama_response(prompt: str) -> None:
    '''
    Generate a response from the Llama model.

    Parameters:
        prompt (str): The user's input/question for the model.

    Returns:
        None: Prints the model's response.
    '''
    sequences = llama_pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=256,
    )
    print("Chatbot:", sequences[0]['generated_text'])
"""

language = "Python"

In [26]:
template = """
Here's my function in {language}:

{function}

Given the definition of a function in any programming language (particularly Python and C++), please generate it's stand-alone documentation. I want it complete with fields like function name, function arguments and return values as well as a detailed explanation of how the function logic works line-by-line. Make it concise and informative to put the documentation into a project documentation file.
"""

In [27]:
# template = """
# Given the definition of a function in any programming language, please generate it's documentation. The documentation needs to include fields like function name, function arguments and return values as well as a detailed explanation of how the function logic works line-by-line.

# This is a function in {language}:
# {function}

# Generate it's documentation in the following format.

# # Name of the function:
# The function definition and body here.
# # Arguments:
# List of arguments along with their types. Display them like this:
# :param n: An integer representing the position of the desired Fibonacci number in the sequence.
# # Return values:
# List of all the values returned by the function with their types. Display them like this.
# :return: The nth Fibonacci number as an integer.
# # Explanation:
# Line-by-line explanation of the function's logic and implementation.
# # Use cases:
# How the function may be used in an intended context. Provide an example with code.
# """

prompt = PromptTemplate(
    template=template,
    input_variables=["language", "function"],
)

In [28]:
llmChain = LLMChain(
    prompt=prompt, 
    llm=llm,
)

In [29]:
print(prompt.format(language=language, function=testFunction))


Here's my function in Python:


def get_llama_response(prompt: str) -> None:
    '''
    Generate a response from the Llama model.

    Parameters:
        prompt (str): The user's input/question for the model.

    Returns:
        None: Prints the model's response.
    '''
    sequences = llama_pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=256,
    )
    print("Chatbot:", sequences[0]['generated_text'])


Given the definition of a function in any programming language (particularly Python and C++), please generate it's stand-alone documentation. I want it complete with fields like function name, function arguments and return values as well as a detailed explanation of how the function logic works line-by-line. Make it concise and informative to put the documentation into a project documentation file.



In [30]:
llmChain.run({"language": language, "function": testFunction})


Function Name: get_llama_response

Description: This function generates a response from the Llama model based on user input.

Arguments:
- prompt (str): The user's input/question for the model.

Returns:
None: Prints the model's response.

Explanation:
The get_llama_response function takes a single argument, prompt, which is of type string. It then uses the llama_pipeline function from the Hugging Face Transformers library to generate a response from the Llama model based on the user's input.

The llama_pipeline function returns a list of dictionaries, where each dictionary represents a generated sequence of tokens by the model. The first element in this list is used as the response for the chatbot.

Finally, the function prints the generated text from the Llama model using the print() function.

Example Usage:
```python
get_llama_response("What's your favorite color?")
```

'\nFunction Name: get_llama_response\r\n\r\nDescription: This function generates a response from the Llama model based on user input.\r\n\r\nArguments:\r\n- prompt (str): The user\'s input/question for the model.\r\n\r\nReturns:\r\nNone: Prints the model\'s response.\r\n\r\nExplanation:\r\nThe get_llama_response function takes a single argument, prompt, which is of type string. It then uses the llama_pipeline function from the Hugging Face Transformers library to generate a response from the Llama model based on the user\'s input.\r\n\r\nThe llama_pipeline function returns a list of dictionaries, where each dictionary represents a generated sequence of tokens by the model. The first element in this list is used as the response for the chatbot.\r\n\r\nFinally, the function prints the generated text from the Llama model using the print() function.\r\n\r\nExample Usage:\r\n```python\r\nget_llama_response("What\'s your favorite color?")\r\n```'